<a href="https://colab.research.google.com/github/Ramzy70/FreeCodeCamp-MachineLearning-Course/blob/main/tensorFlow/8_Natural_Language_Processing_with_RNNs_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Movie Review Dataset
Well start by loading in the IMDB movie review dataset from keras. This dataset contains 25,000 reviews from IMDB where each one is already preprocessed and has a label as either positive or negative. Each review is encoded by integers that represents how common a word is in the entire dataset

In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
train_data[1]

###More Preprocessing
we must make each review the same length.

if the review is greater than 250 words then trim off the extra words

if the review is less than 250 words add the necessary amount of 0's to make it equal to 250.

In [4]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

###Creating the Model

a word embedding layer as the first layer in our model and add a LSTM layer afterwards that feeds into a dense node to get our predicted sentiment.

32 stands for the output dimension of the vectors generated by the embedding layer.

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


###Training

In [7]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 70s 108ms/step - loss: 0.4466 - acc: 0.7846 - val_loss: 0.3192 - val_acc: 0.8646
Epoch 2/10
625/625 [==============================] - 72s 115ms/step - loss: 0.2600 - acc: 0.8999 - val_loss: 0.3419 - val_acc: 0.8498
Epoch 3/10
625/625 [==============================] - 70s 111ms/step - loss: 0.2006 - acc: 0.9257 - val_loss: 0.2946 - val_acc: 0.8744
Epoch 4/10
625/625 [==============================] - 68s 109ms/step - loss: 0.1600 - acc: 0.9428 - val_loss: 0.3032 - val_acc: 0.8786
Epoch 5/10
625/625 [==============================] - 68s 109ms/step - loss: 0.1341 - acc: 0.9554 - val_loss: 0.3002 - val_acc: 0.8862
Epoch 6/10
625/625 [==============================] - 71s 113ms/step - loss: 0.1148 - acc: 0.9627 - val_loss: 0.3461 - val_acc: 0.8838
Epoch 7/10
625/625 [==============================] - 66s 106ms/step - loss: 0.0969 - acc: 0.9676 - val_loss: 0.3820 - val_acc: 0.8832
Epoch 8/10
625/625 [==============================] - 6

In [8]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 22s 28ms/step - loss: 0.4848 - acc: 0.8601
[0.48478981852531433, 0.8601199984550476]


###Making Predictions

need to convert any review that we write into that form so the network can understand it.

In [9]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

1641221/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [10]:
#a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [11]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

1/1 [==============================] - 1s 598ms/step
[0.8246285]
1/1 [==============================] - 0s 31ms/step
[0.08466476]


##RNN Play Generator

We will simply show the RNN an example of something we want it to recreate and it will learn how to write a version of it on its own. We'll do this using a character predictive model that will take as input a variable length sequence and predict the next character.

In [2]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


###Dataset
For this example, we only need one peice of training data. In fact, we can write our own poem or play and pass that to the network for training if we'd like. However, to make things easy we'll use an extract from a shakesphere play.

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# #load our own file :
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

1115394/1115394 [==============================] - 0s 0us/step


###Read Contents of File

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))
# Take a look at the first 250 characters in text
print(text[:250])

Length of text: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



###Encoding

In [5]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [6]:
# lets look at how part of our text is encoded
print("Text:", text[:14])
print("Encoded:", text_to_int(text[:14]))

Text: First Citizen:
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52 10]


In [7]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


###Creating Training Examples

Remember our task is to feed the model a sequence and have it return to us the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples.

The training examples we will prepapre will use a seq_length sequence as input and a seq_length sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

input: Hell | output: ello

Our first step will be to create a stream of characters from our text data.

In [8]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Next we can use the batch method to turn this stream of characters into batches of desired length.

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Now we need to use these sequences of length 101 and split them into input and output.

In [10]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))

Finally we need to make training batches.

In [12]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

###Building the Model

We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


###Creating a Loss Function

This is because our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.

However, before we do that let's have a look at a sample input and the output from our untrained model. This is so we can understand what the model is giving us.

In [14]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [15]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-1.3204047e-03 -3.9539335e-04  4.5113750e-03 ...  2.2294417e-03
    5.8007526e-04  3.1740561e-03]
  [-1.6502654e-03 -6.0496032e-03  4.3706875e-03 ...  6.0818018e-04
   -3.4807909e-03 -2.4972977e-03]
  [ 4.0300610e-04 -3.3050212e-03  3.4007165e-03 ... -5.5692978e-03
    2.1794811e-04 -6.2819980e-03]
  ...
  [ 7.4772825e-03 -2.5157155e-03  2.2856472e-04 ...  6.7156241e-03
   -6.3244454e-03  3.2259426e-03]
  [ 8.2272757e-03 -1.0871575e-03 -3.7072157e-04 ...  1.3362144e-02
   -4.4241967e-03  6.7863902e-03]
  [ 4.4185231e-03 -3.2451302e-03  1.6770585e-03 ...  9.7275246e-03
   -6.8767914e-03  3.8120653e-03]]

 [[ 3.5936662e-03  1.8316724e-03  2.9544970e-03 ...  6.7052236e-03
   -1.6217737e-03 -4.3734588e-05]
  [ 9.5845880e-03 -5.2106073e-03 -7.6291314e-04 ...  3.1869370e-03
   -4.6490245e-03 -4.4642556e-03]
  [ 6.8074758e-03 -5.2315677e-03  5.6304662e-03 ...  3.7252945e-03
   -2.4439285e-03 -7.2635047e-04]
  ...
  [ 6.4669591e-03 -6.1391569e-03  5.8343741e-03 ...  1.4232159e

In [16]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.0013204  -0.00039539  0.00451137 ...  0.00222944  0.00058008
   0.00317406]
 [-0.00165027 -0.0060496   0.00437069 ...  0.00060818 -0.00348079
  -0.0024973 ]
 [ 0.00040301 -0.00330502  0.00340072 ... -0.0055693   0.00021795
  -0.006282  ]
 ...
 [ 0.00747728 -0.00251572  0.00022856 ...  0.00671562 -0.00632445
   0.00322594]
 [ 0.00822728 -0.00108716 -0.00037072 ...  0.01336214 -0.0044242
   0.00678639]
 [ 0.00441852 -0.00324513  0.00167706 ...  0.00972752 -0.00687679
   0.00381207]], shape=(100, 65), dtype=float32)


In [17]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-1.3204047e-03 -3.9539335e-04  4.5113750e-03  2.7483127e-03
  3.1255691e-03 -1.6682585e-03 -1.4908050e-04 -1.4614237e-03
 -5.4662568e-03 -8.7734836e-04  5.1864982e-04  2.4565631e-03
  2.5222381e-03  2.2581869e-03 -4.2818050e-04 -1.7698623e-03
 -2.5108992e-04 -1.6980554e-03  3.0430430e-03 -3.6683180e-03
 -1.3452647e-03 -1.8932562e-03 -3.8073107e-04 -4.8284167e-03
  6.9646775e-03 -1.8946135e-03 -1.6204123e-03  3.4167722e-03
 -1.0488674e-03 -1.7591047e-03  3.1293419e-03 -8.9540554e-04
  3.0250037e-03  3.1528389e-03 -2.5479491e-03 -1.3555789e-03
  1.8852871e-03  5.6874123e-05 -1.3076584e-04  6.1896737e-03
  7.6784572e-04 -1.0240761e-03  1.6525486e-03 -7.7073800e-04
  5.2879676e-03  2.3331216e-03  3.8098139e-03  5.7432335e-05
  1.6277834e-03  1.1331986e-03 -2.9627606e-04  1.5055626e-03
 -3.7647462e-03 -6.6682426e-03 -2.1202364e-03 -1.9823445e-03
  4.0110005e-03 -1.8190159e-03  1.2164454e-03  1.7777251e-03
  2.7726493e-03  2.0906157e-03  2.2294417e-03  5.8007526e-04
  3.174056

In [18]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"FtNONdgKLU Vzr&D:UenxI'$rzoYhbdoM.aCcRRS yD';YW-$oFuiZckfZQv;c-$AQCt'Vq,iSkhJo&WY$LJ&DxFLvSkeLO-3TjH"

So now we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were.

In [19]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

###Compiling the Model

In [20]:
model.compile(optimizer='adam', loss=loss)

###Creating Checkpoints

This will allow us to load our model from a checkpoint and continue training it.

In [21]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

###Training

In [22]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 18s 68ms/step - loss: 2.7048
Epoch 2/50
172/172 [==============================] - 13s 67ms/step - loss: 2.0323
Epoch 3/50
172/172 [==============================] - 14s 68ms/step - loss: 1.7735
Epoch 4/50
172/172 [==============================] - 14s 70ms/step - loss: 1.6234
Epoch 5/50
172/172 [==============================] - 14s 69ms/step - loss: 1.5272
Epoch 6/50
172/172 [==============================] - 14s 69ms/step - loss: 1.4620
Epoch 7/50
172/172 [==============================] - 14s 69ms/step - loss: 1.4138
Epoch 8/50
172/172 [==============================] - 14s 70ms/step - loss: 1.3772
Epoch 9/50
172/172 [==============================] - 14s 70ms/step - loss: 1.3466
Epoch 10/50
172/172 [==============================] - 14s 70ms/step - loss: 1.3197
Epoch 11/50
172/172 [==============================] - 14s 71ms/step - loss: 1.2961
Epoch 12/50
172/172 [==============================] - 14s 72ms/step - loss: 1.2732
E

###Loading the Model

We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one peice of text to the model and have it make a prediction.

In [23]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model is finished training, we can find the lastest checkpoint that stores the models weights using the following line.

In [24]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

##Generating Text

In [25]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [29]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: can i see your tits?
can i see your tits?
Is there were none at all my heart!

First Citizen:
Clarence, be suppers'd,
To entreat his head.
Your brows are full of grief, make them all other be.

Lord Marshal:
My guilt be done,
And let the appellant to thy gird: ay, to yours.

LADY CAPULET:
Ay, and go;
And, sir: what makes these king? my love all res,
Pronounce the villain whose ill hum forbids to Accepory for the

And he shall pay we joy. What, since the death of Angelo, hare!

HORTENSIO:
Faith, a quarrel son!
They'll quickly knaves us rich, most good insten to the king,
To learn of him: these are the reaported feople and Friar Loling on the party,
And, thou goest with true? you must with her,
Should be it to a voice music be, the noisome arguments.

GLOUCESTER:
Boy!

BIRHANDEY:
Romeo! how hast thou it seem,
Nor how to-day, O you 
